In [7]:
# Data Loading and Preprocessing
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, Input, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization # Ensure BatchNormalization is imported

print("--- Data Loading and Preprocessing ---")

def one_hot_encode_sequence(sequence, max_len):
    mapping = {
        'A': [1, 0, 0, 0], 'a': [1, 0, 0, 0],
        'T': [0, 1, 0, 0], 't': [0, 1, 0, 0],
        'G': [0, 0, 1, 0], 'g': [0, 0, 1, 0],
        'C': [0, 0, 0, 1], 'c': [0, 0, 0, 1],
        'N': [0, 0, 0, 0], 'n': [0, 0, 0, 0]
    }
    encoded = [mapping.get(char, [0, 0, 0, 0]) for char in sequence]
    encoded = np.array(encoded, dtype=np.float32)
    if len(encoded) < max_len:
        pad = np.zeros((max_len - len(encoded), 4), dtype=np.float32)
        encoded = np.vstack((encoded, pad))
    return encoded[:max_len]

train_df = pd.read_csv('dm3.kc167.tads.train.csv', header=None)
X_train_raw, y_train = train_df[0].values, train_df[1].values
test_df = pd.read_csv('dm3.kc167.tads.test.csv', header=None)
X_test_raw, y_test = test_df[0].values, test_df[1].values

sequence_length = len(X_train_raw[0])
X_train = np.array([one_hot_encode_sequence(seq, sequence_length) for seq in X_train_raw])
X_test = np.array([one_hot_encode_sequence(seq, sequence_length) for seq in X_test_raw])
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
input_shape = (sequence_length, 4)

# --- Global Training Parameters (Less about model capacity, more about training process) ---
# These are kept consistent for all models to standardize the training procedure.
LEARNING_RATE = 0.001 # Set to the lowest suggested learning rate for stability and adherence to problem.
EPOCHS = 100 # Still give enough epochs for convergence, relying on early stopping.
BATCH_SIZE = 64 # Good balance for CPU.
VALIDATION_SPLIT = 0.2 # Standard validation split.
DROPOUT_RATE_COMMON = 0.3 # A common, lower dropout rate for all models to speed up initial learning.

# --- Callbacks ---
# Patience for early stopping is increased to give models more chance to learn with reduced capacity.
# ReduceLROnPlateau remains a crucial callback for stable training.
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)

print("\n--- Training Models ---")

--- Data Loading and Preprocessing ---
Train shape: (28140, 1000, 4) Test shape: (2000, 1000, 4)

--- Training Models ---


In [8]:
# Model 1 	Model 1 – Simple CNN
# Optimized for speed with smaller kernels and fewer filters, within suggested range.
print("\n--- Model 1: Simple CNN ---")
model1 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced filters & kernel size
    MaxPooling1D(pool_size=2),
    Dropout(DROPOUT_RATE_COMMON),
    Flatten(),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model1.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model1.evaluate(X_test, y_test, verbose=0)
print(f"Model 1 Test Accuracy: {accuracy:.4f}")



--- Model 1: Simple CNN ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5400 - loss: 0.7620 - val_accuracy: 0.0000e+00 - val_loss: 1.1226 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6318 - loss: 0.6540 - val_accuracy: 0.0000e+00 - val_loss: 1.0254 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6481 - loss: 0.6421 - val_accuracy: 0.1135 - val_loss: 1.0710 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7124 - loss: 0.5775 - val_accuracy: 0.7194 - val_loss: 0.5603 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7442 - loss: 0.5379 - val_accuracy: 0.5215 - val_loss: 0.7038 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.7699 - loss: 0.5014 - val_accuracy: 0.6910 - val_loss: 0.5512 - learning_rate: 0.0010
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━

In [9]:
# Model 2 Model 2 – Deeper CNN
# Significantly reduced filters and kernel size for deeper CNN to manage CPU load.
print("\n--- Model 2: Deeper CNN ---")
model2 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=16, kernel_size=9, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced filters & kernel size
    MaxPooling1D(pool_size=2), Dropout(DROPOUT_RATE_COMMON),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced filters
    MaxPooling1D(pool_size=2), Dropout(DROPOUT_RATE_COMMON),
    Flatten(),
    Dense(units=64, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model2.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print(f"Model 2 Test Accuracy: {accuracy:.4f}")



--- Model 2: Deeper CNN ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.5457 - loss: 0.7739 - val_accuracy: 0.0107 - val_loss: 0.9346 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6148 - loss: 0.6719 - val_accuracy: 0.0000e+00 - val_loss: 1.0700 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6599 - loss: 0.6249 - val_accuracy: 0.3564 - val_loss: 0.8545 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6972 - loss: 0.5872 - val_accuracy: 0.7434 - val_loss: 0.5789 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.7173 - loss: 0.5722 - val_accuracy: 0.2168 - val_loss: 1.0125 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.7234 - loss: 0.5621 - val_accuracy: 0.0311 - val_loss: 1.4579 - learning_rate: 0.0010
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 6s

In [10]:
# Model 3 	Model 3 – Simple LSTM
# Minimum LSTM neurons as suggested to reduce computation.
print("\n--- Model 3: Simple LSTM ---")
model3 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10)), # Min suggested LSTM neurons for significant speedup
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model3.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model3.evaluate(X_test, y_test, verbose=0)
print(f"Model 3 Test Accuracy: {accuracy:.4f}")


--- Model 3: Simple LSTM ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 42s 115ms/step - accuracy: 0.5459 - loss: 0.7426 - val_accuracy: 0.0000e+00 - val_loss: 0.9789 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 48s 137ms/step - accuracy: 0.6079 - loss: 0.6739 - val_accuracy: 0.0000e+00 - val_loss: 0.9628 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 49s 138ms/step - accuracy: 0.6249 - loss: 0.6679 - val_accuracy: 0.0000e+00 - val_loss: 0.9916 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 49s 138ms/step - accuracy: 0.6212 - loss: 0.6664 - val_accuracy: 0.0000e+00 - val_loss: 0.9762 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 48s 137ms/step - accuracy: 0.6266 - loss: 0.6624 - val_accuracy: 5.3305e-04 - val_loss: 0.9807 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 49s 138ms/step - accuracy: 0.6251 - loss: 0.6626 - val_accuracy: 5.3305e-04 - val_loss: 0.9768 - learning_rate: 0.0010
Epoch 7/10

In [11]:
# Model 4 Model 4 – Deeper LSTM
# Minimum LSTM neurons for each layer to make training feasible on CPU.
print("\n--- Model 4: Deeper LSTM ---")
model4 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10, return_sequences=True)), # Min suggested LSTM neurons
    Dropout(DROPOUT_RATE_COMMON),
    Bidirectional(LSTM(units=10)), # Min suggested LSTM neurons
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=64, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model4.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model4.evaluate(X_test, y_test, verbose=0)
print(f"Model 4 Test Accuracy: {accuracy:.4f}")


--- Model 4: Deeper LSTM ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 94s 262ms/step - accuracy: 0.5751 - loss: 0.7070 - val_accuracy: 0.0000e+00 - val_loss: 0.9845 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 99s 282ms/step - accuracy: 0.6217 - loss: 0.6651 - val_accuracy: 0.0000e+00 - val_loss: 0.9781 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 99s 282ms/step - accuracy: 0.6263 - loss: 0.6630 - val_accuracy: 0.0000e+00 - val_loss: 0.9871 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 100s 284ms/step - accuracy: 0.6249 - loss: 0.6630 - val_accuracy: 0.0000e+00 - val_loss: 0.9632 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 100s 283ms/step - accuracy: 0.6249 - loss: 0.6629 - val_accuracy: 0.0000e+00 - val_loss: 0.9706 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 100s 284ms/step - accuracy: 0.6200 - loss: 0.6650 - val_accuracy: 0.0000e+00 - val_loss: 0.9963 - learning_rate: 0.0010
Epoch 7

KeyboardInterrupt: 

In [ ]:
# Model 5 CNN-LSTM Hybrid
# Optimized CNN and LSTM parameters for speed.
print("\n--- Model 5: CNN-LSTM Hybrid ---")
model5 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced filters & kernel size
    MaxPooling1D(pool_size=2), Dropout(DROPOUT_RATE_COMMON),
    Bidirectional(LSTM(units=10)), # Min suggested LSTM neurons
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model5.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model5.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model5.evaluate(X_test, y_test, verbose=0)
print(f"Model 5 Test Accuracy: {accuracy:.4f}")



--- Model 5: CNN-LSTM Hybrid ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 30s 81ms/step - accuracy: 0.5548 - loss: 0.7330 - val_accuracy: 0.0000e+00 - val_loss: 1.0023 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 31s 88ms/step - accuracy: 0.6166 - loss: 0.6701 - val_accuracy: 0.0000e+00 - val_loss: 0.9567 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 31s 87ms/step - accuracy: 0.6256 - loss: 0.6625 - val_accuracy: 0.0000e+00 - val_loss: 1.0067 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 31s 88ms/step - accuracy: 0.6300 - loss: 0.6598 - val_accuracy: 0.0000e+00 - val_loss: 1.0048 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 31s 88ms/step - accuracy: 0.6201 - loss: 0.6630 - val_accuracy: 0.1725 - val_loss: 0.8029 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 32s 91ms/step - accuracy: 0.6233 - loss: 0.6593 - val_accuracy: 0.0000e+00 - val_loss: 1.1728 - learning_rate: 0.0010
Epoch 7/100
352/

KeyboardInterrupt: 

In [ ]:
# Model 6 CNN with More Dense Layers
# Optimized CNN parameters for speed.
print("\n--- Model 6: CNN with More Dense Layers ---")
model6 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced filters & kernel size
    MaxPooling1D(pool_size=2), Dropout(DROPOUT_RATE_COMMON), Flatten(),
    Dense(units=64, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model6.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model6.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model6.evaluate(X_test, y_test, verbose=0)
print(f"Model 6 Test Accuracy: {accuracy:.4f}")


In [ ]:
# Model 7 LSTM with More Dense Layers
# Minimum LSTM neurons for significant speedup.
print("\n--- Model 7: LSTM with More Dense Layers ---")
model7 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10)), # Min suggested LSTM neurons
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=64, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    Dense(units=32, activation='relu', kernel_regularizer=l2(0.00005)), # Reduced Dense neurons
    Dropout(DROPOUT_RATE_COMMON),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model7.compile(optimizer=Adam(LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])
model7.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SPLIT,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model7.evaluate(X_test, y_test, verbose=0)
print(f"Model 7 Test Accuracy: {accuracy:.4f}")